In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import gzip
import ast
import pandas as pd

reviews = []

with open ('/content/drive/My Drive/PI MLOps - STEAM/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))

df_reviews = pd.DataFrame(reviews)


In [ ]:
df_reviewsOpen = df_reviews.explode('reviews')
df_reviewsOpen = pd.concat([df_reviewsOpen.drop('reviews', axis=1), df_reviewsOpen['reviews'].apply(pd.Series)], axis=1)


In [ ]:
#Este codigo elimina columnas innecesarias para el analisis
eliminar_columnas = ['user_url', 'funny', 'posted', 'last_edited', 'helpful']
df_reviewsOpen = df_reviewsOpen.drop(eliminar_columnas, axis=1)

In [ ]:
df_reviewsOpen = df_reviewsOpen.dropna(axis=1, how='all')

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Descargar los recursos necesarios para NLTK (solo necesario la primera vez)
nltk.download('vader_lexicon')

# Crear un analizador de sentimiento
sia = SentimentIntensityAnalyzer()

# Función para asignar el análisis de sentimiento o el valor predeterminado
def analyze_sentiment_or_default(text):
    if isinstance(text, str):
        compound_score = sia.polarity_scores(text)['compound']
        if compound_score >= 0.05:
            return 2
        elif compound_score <= -0.05:
            return 0
        else:
            return 1
    else:
        return 1

# Aplicar análisis de sentimiento o valor predeterminado y crear la nueva columna 'sentiment_analysis'
df_reviewsOpen['sentiment_analysis'] = df_reviewsOpen['review'].apply(analyze_sentiment_or_default)

# Reemplazar la columna 'reviews' por la nueva 'sentiment_analysis'
df_reviewsOpen.drop(columns=['review'], inplace=True)

# Imprimir el DataFrame resultante
print(df_reviewsOpen)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


                 user_id item_id recommend  sentiment_analysis
0      76561197970982479    1250      True                   2
0      76561197970982479   22200      True                   2
0      76561197970982479   43110      True                   2
1                js41637  251610      True                   2
1                js41637  227300      True                   2
...                  ...     ...       ...                 ...
25797  76561198312638244      70      True                   2
25797  76561198312638244  362890      True                   2
25798        LydiaMorley  273110      True                   2
25798        LydiaMorley     730      True                   2
25798        LydiaMorley     440      True                   2

[59333 rows x 4 columns]


In [ ]:
print(df_reviewsOpen.isnull().sum())

user_id                0
item_id               28
recommend             28
sentiment_analysis     0
dtype: int64


In [ ]:
ruta_guardar = '/content/drive/My Drive/PI MLOps - STEAM/Review.csv'
df_reviewsOpen.to_csv(ruta_guardar, index=False)